## DON'T re run this notebook !

### This is a notebook that contains code that allows us to compute the perplexity of a model. This was set up on a shared drive. It's better just to look at the printed results. 

#### We can note that this an additional task we tried to do but it didn't work as expected. We still submit it for the project as this is something we worked on, even if we can't use it for our final results.

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
torch.cuda.empty_cache()

True
1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AutoModelWithLMHead, AutoTokenizer

device = "cuda"
model_id = "gpt2-large"
model = AutoModelWithLMHead.from_pretrained('/content/drive/Shareddrives/ML4Science/gpt2-poetry-model-with-start-token_100_epochs').to(device)

#model = AutoModelWithLMHead.from_pretrained('/content/drive/MyDrive/Colab Notebooks/gpt2-poetry-model-crpo').to(device)
#model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
#model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/Colab Notebooks/Calculate perplexity.ipynb').to(device)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")#AutoTokenizer.from_pretrained("gpt2")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 30.8 MB/s 
     |████████████████████████████████| 182 kB 73.9 MB/s 
     |████████████████████████████████| 7.6 MB 55.6 MB/s 


/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
!pip install datasets
from datasets import load_dataset
#test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
#test = []
read_file = "/content/drive/MyDrive/Colab Notebooks/rhyming_poems_10_epoch_task1.txt"
#with open(read_file, 'r', encoding='utf-8') as f:
 #   for line in f.readlines():
  #      test.append(line)
#test = load_dataset("txt","../rhymedata/english_raw/rhyming_poems_100_epoch_task1", split='test')
test = load_dataset("text", data_files={"test": read_file}, split="test")
#blip = " ".join(test)
print(test)
encodings = tokenizer("\n".join(test["text"]), return_tensors="pt")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Dataset({
    features: ['text'],
    num_rows: 2227
})


In [ ]:
"\n\n".join(test["text"])

"The taste of marmelade is better with you\n\n You light up my day when you lace your shoe\n\n\n\n\n\nOn that he had in mind to wait\n\nAnd at the first and last in grate\n\n\n\nHe drew a wicket, and at last\n\nThat man within was left behind\n\n\n\nFor, with the sun and darkness all\n\nThe great and faithful dead are all\n\n\n\nThe last great dead that live and die\n\nIs the great one dead. Then he lye\n\n\n\nThe nightwind is my heart! it stirs\n\nA great and silent world of soprano's\n\n\n\nI am a spirit in the skies\n\n'T is night before my golden day's\n\n\n\nWhen, on each cheek, like summertints shed\n\nThe crimson crimson sunshine of the spray\n\n\n\nOr, in her arms, a maiden in her arms\n\nOr in her heart a lover's charms\n\n\n\nOr in her eyes a mother's words, or mirth's\n\n\n\nTo such a blissful world of woe and bliss\n\n\n\n\n\nThat, if she could, she should, or I should die or I might try\n\nWith me a paean\n\n\n\nAye my life and soul like a dream of this aye\n\n\n\n\n\nThen

In [ ]:
import torch
from tqdm import tqdm

max_length = model.config.n_positions
stride = 1024
seq_len = encodings.input_ids.size(1)

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over input tokens.
        # Multiply it with trg_len to get the summation instead of average.
        # We will take average over all the tokens to get the true average
        # in the last step of this example.
        neg_log_likelihood = outputs.loss * trg_len

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break
    print(torch.exp(torch.stack(nlls).sum() / end_loc))

ppl = torch.exp(torch.stack(nlls).sum() / end_loc)

 13%|█▎        | 2/15 [00:00<00:02,  5.86it/s]

tensor(64139.1680, device='cuda:0')
tensor(116362.6562, device='cuda:0')


 27%|██▋       | 4/15 [00:00<00:01,  7.04it/s]

tensor(81993.8672, device='cuda:0')
tensor(98064.5781, device='cuda:0')


 40%|████      | 6/15 [00:00<00:01,  8.14it/s]

...

tensor(79038.0156, device='cuda:0')
tensor(76998.8594, device='cuda:0')


 80%|████████  | 12/15 [00:01<00:00,  8.21it/s]

tensor(85338.9609, device='cuda:0')
tensor(82635.6875, device='cuda:0')


 93%|█████████▎| 14/15 [00:01<00:00,  7.70it/s]

tensor(89932.8281, device='cuda:0')
tensor(86613.1094, device='cuda:0')


In [ ]:
ppl

tensor(84486.5000, device='cuda:0')

In [ ]:
max_length

1024